# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



*Reading the Crunchbase data in chunks of 5000 rows, then print the first 5 rows of each chunk. We need to see the data and make sure that the chunks are being read correctly by pandas. The encoding parameter is set to 'unicode_escape' to properly handle any special characters in the data.*

In [ ]:
# Your code goes here
url = "https://bit.ly/3BPcobU"
#
import pandas as pd
for crunchbase in  pd.read_csv(url, encoding='unicode_escape', chunksize = 5000):
  print(crunchbase.head(5))



     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

*We want to return the memory usage of each column in the current chunk of the Crunchbase data. The deep parameter has been set to True, which means that the memory usage of all the elements in each column will be calculated, including any nested elements. The result is a pandas Series object, where the index is the column names and the values are the memory usage of each column in bytes.*

In [ ]:
crunchbase.memory_usage(deep = True)

Index                        132
company_permalink         215412
company_name              189709
company_category_code     184581
company_country_code      172200
company_state_code        168547
company_region            184997
company_city              191495
investor_permalink        222759
investor_name             199818
investor_category_code     22960
investor_country_code      22960
investor_state_code        22960
investor_region           183680
investor_city              22960
funding_round_type        182367
funded_at                 192290
funded_month              183680
funded_quarter            183680
funded_year                22960
raised_amount_usd          22960
dtype: int64

*We intend to calculate the total memory footprint of all the chunks of the Crunchbase data by summing up the memory usage of each chunk. The memory_usage() method is called on each chunk to get its memory usage, and the .sum() method is used to get the sum of all the memory usage of each column in the chunk. The result is then divided by 1024 * 1024 to convert it from bytes to megabytes. Finally, the memory usage of each chunk is appended to the memory_footprints list, and the sum of all the values in memory_footprints is printed.*
                                                                                                                                                   *This gives us the combined output*

In [ ]:
memory_footprints = []
crunchbase_iter = pd.read_csv(url, encoding='unicode_escape', chunksize = 5000)
for chunk in crunchbase_iter:
    memory_footprints.append(crunchbase.memory_usage(deep=True).sum()/(1024*1024))
print(sum(memory_footprints))

29.30085849761963


*To remove the unnecessary columns, we read the entire Crunchbase data into a single pandas DataFrame, and then drop several columns that aren't useful for analysis. The drop method is used to remove the specified columns, which are listed in the argument passed to the columns parameter.
The axis parameter is set to 1 to indicate that we are dropping columns (as opposed to rows), and the result of this operation is 
a new DataFrame with the specified columns removed. Note that this code doesn't assign the result of the drop operation back to the crunchbase DataFrame. To make the change permanent, you should re-assign the result to crunchbase like this: crunchbase = crunchbase.drop(['investor_country_code', 'investor_state_code','investor_category_code', 'company_city','raised_amount_usd','investor_city', 'company_state_code','company_category_code',], axis = 1).*

In [ ]:
crunchbase = pd.read_csv(url, encoding='unicode_escape')
crunchbase.drop(['investor_country_code', 'investor_state_code','investor_category_code', 'company_city','raised_amount_usd','investor_city','company_state_code','company_category_code',], axis = 1)
     

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,company_permalink,company_name,company_country_code,company_region,investor_permalink,investor_name,investor_region,funding_round_type,funded_at,funded_month,funded_quarter,funded_year
0,/company/advercar,AdverCar,USA,SF Bay,/company/1-800-flowers-com,1-800-FLOWERS.COM,New York,series-a,2012-10-30,2012-10,2012-Q4,2012.0
1,/company/launchgram,LaunchGram,USA,SF Bay,/company/10xelerator,10Xelerator,Columbus,other,2012-01-23,2012-01,2012-Q1,2012.0
2,/company/utap,uTaP,USA,United States - Other,/company/10xelerator,10Xelerator,Columbus,other,2012-01-01,2012-01,2012-Q1,2012.0
3,/company/zoopshop,ZoopShop,USA,Columbus,/company/10xelerator,10Xelerator,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012.0
4,/company/efuneral,eFuneral,USA,Cleveland,/company/10xelerator,10Xelerator,Columbus,other,2011-09-08,2011-09,2011-Q3,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...
52865,/company/garantia-data,Garantia Data,USA,SF Bay,/person/zohar-gilon,Zohar Gilon,unknown,series-a,2012-08-08,2012-08,2012-Q3,2012.0
52866,/company/duda-mobile,DudaMobile,USA,SF Bay,/person/zohar-gilon,Zohar Gilon,unknown,series-c+,2013-04-08,2013-04,2013-Q2,2013.0
52867,/company/sitebrains,SiteBrains,USA,SF Bay,/person/zohar-israel,zohar israel,unknown,angel,2010-08-01,2010-08,2010-Q3,2010.0
52868,/company/comprehend-systems,Comprehend Systems,USA,SF Bay,/person/zorba-lieberman,Zorba Lieberman,unknown,series-a,2013-07-11,2013-07,2013-Q3,2013.0


*We then print the number of missing values for each column in the crunchbase DataFrame. The isnull() method is used to create a DataFrame of the same shape as crunchbase with True values where there are missing values and False values where there are no missing values. The sum method is then applied to this DataFrame to count the number of True values in each column, which gives the number of missing values in each column. The result is a pandas Series object, where the index is the column names and the values are the number of missing values in each column.*

In [ ]:

print( crunchbase.isnull().sum())

company_permalink             1
company_name                  1
company_category_code       643
company_country_code          1
company_state_code          492
company_region                1
company_city                533
investor_permalink            2
investor_name                 2
investor_category_code    50427
investor_country_code     12001
investor_state_code       16809
investor_region               2
investor_city             12480
funding_round_type            3
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
raised_amount_usd          3599
dtype: int64


## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


*Let us use the info() method of a pandas dataframe to display information about the dataframe including:
i. The number of rows and columns
ii. The data types of each column
iii. The memory usage of the dataframe (by column)
This information is useful for understanding the structure of the data and identifying any issues with the data types, such as data being stored as the wrong data type, or columns that take up more memory than necessary.*

In [ ]:
# Your code goes here

print(crunchbase.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52870 entries, 0 to 52869
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       52869 non-null  object 
 1   company_name            52869 non-null  object 
 2   company_category_code   52227 non-null  object 
 3   company_country_code    52869 non-null  object 
 4   company_state_code      52378 non-null  object 
 5   company_region          52869 non-null  object 
 6   company_city            52337 non-null  object 
 7   investor_permalink      52868 non-null  object 
 8   investor_name           52868 non-null  object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   40869 non-null  object 
 11  investor_state_code     36061 non-null  object 
 12  investor_region         52868 non-null  object 
 13  investor_city           40390 non-null  object 
 14  funding_round_type      52867 non-null

*First we create a list numeric that includes the data types corresponding to numeric data (integer and floating point types). Then we use the select_dtypes method to extract the columns in the dataframe crunchbase that have data types included in numeric, and stores the resulting column names in a variable numeric_cols. Finally, the numeric_cols variable is printed to display the names of the numeric columns.*

In [ ]:
numeric= ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = crunchbase.select_dtypes(include=numeric).columns
numeric_columns

Index(['funded_year', 'raised_amount_usd'], dtype='object')

*We then iterate over the chunks of the crunchbase-investments.csv file and for each chunk, it calculates the value counts of the unique values in each column of the chunk. The value counts are stored in the chunk_vc variable and printed for each iteration.*

In [ ]:
crunchbase_iter = pd.read_csv(url, encoding='unicode_escape', chunksize = 5000)
for crunchbase in crunchbase_iter:
    chunk_vc = crunchbase.value_counts()
    print(chunk_vc)

company_permalink                           company_name                       company_category_code  company_country_code  company_state_code  company_region  company_city   investor_permalink                 investor_name             investor_category_code  investor_country_code  investor_state_code  investor_region  investor_city  funding_round_type  funded_at   funded_month  funded_quarter  funded_year  raised_amount_usd
/company/massachusetts-clean-energy-center  Massachusetts Clean Energy Center  cleantech              USA                   MA                  Boston          Boston         /company/north-shore-innoventures  North Shore InnoVentures  biotech                 USA                    MA                   Boston           Beverly        angel               2013-09-05  2013-09       2013-Q3         2013         300000.0             2
/company/1000memories                       1000memories                       web                    USA                   CA           

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

*We read the csv file in chunks of 5000 rows at a time using pandas' read_csv method. We then connect to a SQLite database with the connection object "conn", and for each chunk, we write the chunk to an SQLite table called "exhibitions". After writing each chunk, we run a SQL query "PRAGMA table_info(exhibitions);" to retrieve the table information for the "exhibitions" table, and store the results in a Pandas dataframe called "results_df". Finally, we prints the "results_df" dataframe.*

In [ ]:
# Your code goes here
# Your code goes here
import sqlite3
import pandas as pd
crunchbase_iter = pd.read_csv(url, encoding='unicode_escape', chunksize = 5000)
conn = sqlite3.connect('crunchbase_iter')
for chunk in crunchbase_iter:
    chunk.to_sql("exhibitions", conn, if_exists='append', index=False)
    results_df = pd.read_sql('PRAGMA table_info(exhibitions);', conn)
print(results_df)


    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

# What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

Total amount of funds raised by the top 10% and top 1%

*We execute an SQL query on the crunchbase_iter SQLite database, which we have been reading the data from a CSV file into in chunks and storing in the database. We then select the raised_amount_usd column and count the number of occurrences of each unique value in the raised_amount_usd column and store the results in the eid_counts DataFrame. The query groups the data by raised_amount_usd and orders the results by counts in descending order. Finally, it prints the first 10 rows of the eid_counts DataFrame.*

In [ ]:
# Your code goes here
question = 'select raised_amount_usd, count(*) as counts from exhibitions group by raised_amount_usd  order by counts desc;'
eid_counts = pd.read_sql(question, conn)
print(eid_counts[:10])



   raised_amount_usd  counts
0                NaN    3599
1         10000000.0    2067
2          1000000.0    1760
3          5000000.0    1589
4         15000000.0    1420
5         20000000.0    1312
6          2000000.0    1253
7          1500000.0    1247
8          3000000.0    1204
9          6000000.0    1148


Total amount of funds raised by the bottom 10% and bottom 1%

*We connect to the SQLite database and use a pandas DataFrame to store the results of the SQL query. We store the query in the variable 'question', which selects the 'raised_amount_usd' column and counts the number of rows for each unique value in that column. The result is stored in a pandas DataFrame called 'eid_counts', which is sorted in ascending order by the count of rows for each unique 'raised_amount_usd' value. Finally, we print the first 10 rows of the 'eid_counts' DataFrame.*

In [ ]:
question = 'select raised_amount_usd , count(*) as counts from exhibitions group by raised_amount_usd  order by counts asc;'
eid_counts = pd.read_sql(question, conn)
print(eid_counts[:10])
     

   raised_amount_usd  counts
0             2100.0       1
1             5500.0       1
2             5600.0       1
3             6000.0       1
4             8300.0       1
5            11600.0       1
6            11800.0       1
7            13200.0       1
8            15282.0       1
9            16500.0       1


# Which category of company attracted the most investments?

Top 5 companies with the largest investments

*We execute an SQL query using the pd.read_sql function from the pandas library. The query is selecting the company_category_code column and counting the number of occurrences of each unique value in the column. The result is grouped by the raised_amount_usd column and ordered by the count in descending order to identify the best companies from the top. The resulting data is then displayed with the first 10 rows printed to the console using the print function.*

In [ ]:
query = 'select company_category_code , count(*) as counts from exhibitions group by raised_amount_usd  order by counts desc;'
eid_counts = pd.read_sql(query, conn)
print(eid_counts[:5])

  company_category_code  counts
0                travel    3599
1             cleantech    2067
2                   web    1760
3                mobile    1589
4               biotech    1420


# Which investor contributed the most money (across all startups)?

*We execute an SQL query on the SQLite database created, selecting the "investor_name" column and counting the number of occurrences of each unique value of investor name (grouped by the "raised_amount_usd" and "company_name" columns). The result is ordered by the count in descending order. Finally, we print the first 10 rows of the resulting DataFrame.*

In [ ]:
q = 'select investor_name , count(*) as counts from exhibitions group by raised_amount_usd and company_name order by counts desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

                    investor_name  counts
0               1-800-FLOWERS.COM   52528
1  American Pioneer Ventures, Ltd     321
2                      Amplify.LA      21


# Which investors contributed the most money per startup?

*We execute an SQL query on a database connection conn and retrieve the results of the query into a pandas dataframe eid_counts. The query selects the investor_name, raised_amount_usd, and the count of records with the same raised_amount_usd, and groups the results by raised_amount_usd and orders them in descending order. We then print the first 10 rows of the resulting dataframe.*

In [ ]:
query = 'select investor_name,raised_amount_usd , count(*) as counts from exhibitions group by raised_amount_usd order by raised_amount_usd  desc;'
eid_counts = pd.read_sql(query, conn)
print(eid_counts[:10])

               investor_name  raised_amount_usd  counts
0                BrightHouse       3.200000e+09       5
1     Marlin Equity Partners       2.600000e+09       1
2                BrightHouse       1.500000e+09       8
3                GI Partners       1.050000e+09       2
4                     Google       1.000000e+09       1
5        Andreessen Horowitz       9.500000e+08      10
6                    Comcast       9.200000e+08       5
7       Siemens PLM Software       7.500000e+08       1
8              Goldman Sachs       5.650000e+08       1
9  U.S. Department of Energy       5.350000e+08       1


# Which funding round was the most popular? Which was the least popular?

Most popular funding round

*We are executing an SQL query and retrieving the results of the query into a pandas dataframe eid_counts. The query selects the funding_round_type and the count of records with the same funding_round_type, and groups the results by funding_round_type and orders them in descending order by the count of records. We then prints the first 10 rows of the resulting dataframe.*

In [ ]:
query = 'select funding_round_type , count(*) as counts from exhibitions group by funding_round_type order by counts desc;'
eid_counts = pd.read_sql(query, conn)
print(eid_counts[:10])

  funding_round_type  counts
0           series-a   13938
1          series-c+   10870
2              angel    8989
3            venture    8917
4           series-b    8794
5              other     964
6     private-equity     357
7           post-ipo      33
8       crowdfunding       5
9               None       3


Least popular funding round



*We are executing an SQL query on a database and retrieving the results of the query into a pandas dataframe eid_counts. The query selects the funding_round_type and the count of records with the same funding_round_type, and groups the results by funding_round_type and orders them in ascending order by the count of records. We then prints the first 10 rows of the resulting dataframe.*

In [ ]:
query = 'select funding_round_type , count(*) as counts from exhibitions group by funding_round_type order by counts asc;'
eid_counts = pd.read_sql(query, conn)
print(eid_counts[:10])

  funding_round_type  counts
0               None       3
1       crowdfunding       5
2           post-ipo      33
3     private-equity     357
4              other     964
5           series-b    8794
6            venture    8917
7              angel    8989
8          series-c+   10870
9           series-a   13938
